The matrix given here is 

In [156]:
import numpy as np

A = np.array([[25., -1., 3., 1.], [1., 23., -1., 5.], [3., 1., 25., -1.], [-1., 5., 1., 23.]])
print(A)

[[25. -1.  3.  1.]
 [ 1. 23. -1.  5.]
 [ 3.  1. 25. -1.]
 [-1.  5.  1. 23.]]


First of all, we need to find an eigenvalue with its corresponding eigenvector:

In [182]:
import scipy.linalg as lg

# Return an eigenvalue and one of its eigenvectors
def eigenval_eigenvec(A):
    eigenvalues, eigenvectors = lg.eig(A)
    value = eigenvalues[1]
    print('The eigenvalues are ', np.round(eigenvalues.real,5))
    print('The eigenvectors are \n', np.round(eigenvectors.real,5))
    vector = eigenvectors[:, 1]
    return value, vector

value, vector = eigenval_eigenvec(A)
print('The chosen eigenvalue is ', np.round(value, 5))
print('The corrsponding eigenvector is ', np.round(vector, 5))

The eigenvalues are  [28. 20. 20. 28.]
The eigenvectors are 
 [[ 0.70711  0.5      0.5     -0.13393]
 [ 0.       0.5      0.5      0.69431]
 [ 0.70711 -0.5     -0.5     -0.13393]
 [ 0.      -0.5     -0.5      0.69431]]
The chosen eigenvalue is  (20+0j)
The corrsponding eigenvector is  [ 0.5+0.j  0.5-0.j -0.5-0.j -0.5+0.j]


So the chosen eigenvalue is $20$, and the eigenvector is
\begin{equation}
\begin{bmatrix}
0.5 & 0.5 & -0.5 & -0.5
\end{bmatrix}^T.
\end{equation}
Next let us extend the eigenvector to an orthogonal basis, but we find a general basis first:

In [77]:
# Extend a vector to a basis
def extended_basis(vector): 
    n = len(vector)
    space = []
    space.append(vector)
    for i in range(n - 1):
        arr = []
        for j in range(i):
            arr.append(0)
        arr.append(vector[n-1])
        for j in range(i + 1, n - 1):
            arr.append(0)
        arr.append(-vector[i])
        space.append(arr)
    space = np.transpose(np.array(space))
    return space

basis = entended_basis(vector)
print('The extended basis is \n', np.round(basis, 5))

The extended basis is 
 [[ 0.5+0.j -0.5+0.j  0. +0.j  0. +0.j]
 [ 0.5-0.j  0. +0.j -0.5+0.j  0. +0.j]
 [-0.5-0.j  0. +0.j  0. +0.j -0.5+0.j]
 [-0.5+0.j -0.5-0.j -0.5+0.j  0.5+0.j]]


So the genetal basis we got is
\begin{bmatrix}
0.5 & -0.5 & 0 & 0\\
0.5&0&-0.5&0\\
-0.5 & 0 & 0 & -0.5\\
-0.5 & -0.5 & -0.5 & 0.5
\end{bmatrix}

We find the orthogonal basis by Gram-Schmidt algorithm:

In [196]:
# find the L2 norm of the given vector
def _l2_norm(v):
    sum_ = 0
    for entry in v:
        sum_ += entry ** 2
    return np.sqrt(sum_)

# Apply gram schmidt to make a basis orthogonal
def gram_schmidt(basis):
    n = len(basis)
    for i in range(1, n):
        u = basis[:, i]
        for j in range(0, i):
            ######################################################
            # Change np.dot() to np.vdot() for complex dot product
            ######################################################
            u -= np.dot(u, basis[:, j]) * basis[:, j]
        u = u / _l2_norm(u)
        basis[:, i] = u
    return basis
    
ortho_basis = gram_schmidt(basis)
print(np.round(ortho_basis, 5))

[[ 0.5    +0.j -0.70711+0.j  0.40825+0.j -0.28868-0.j]
 [ 0.5    -0.j  0.     -0.j -0.8165 -0.j -0.28868-0.j]
 [-0.5    -0.j -0.     +0.j -0.     -0.j -0.86603+0.j]
 [-0.5    +0.j -0.70711-0.j -0.40825+0.j  0.28868+0.j]]


Thus, the orthogonal basis we found is
\begin{bmatrix}
0.5 & -0.70711 & 0.40825 & -0.28868\\
0.5 & 0 & -0.8165 & -0.28868\\
-0.5 & 0 & 0 & -0.86603\\
-0.5 & -0.70711 & -0.40825 & 0.28868\\
\end{bmatrix}

Denote the orthogonal basis as $I_1$, then we need to compute $I_1^{-1}$. Since it is orthogonal, we just need to find its transpose. Then we can find $I_1^*A I_1$:

In [138]:
def multiply(I, A):
    ######################################################
    # Change np.dot() to np.vdot() for complex dot product
    ######################################################
    A1 = np.transpose(I).dot(A).dot(I)
    I1 = I
    return A1

A1 = multiply(ortho_basis, A)
print(np.round(A1, 5))

[[20.     -0.j -2.82843+0.j  1.63299+0.j  2.3094 -0.j]
 [ 0.     +0.j 24.     -0.j  2.3094 +0.j  3.26599-0.j]
 [ 0.     -0.j  2.3094 +0.j 26.66667-0.j -1.88562-0.j]
 [ 0.     -0.j  3.26599-0.j -1.88562-0.j 25.33333+0.j]]


So we got $A_1$ as above. Then we can take the submatrix, and repeat the process to get $A_2$ and $I_2$. For our convenience, let us put together the functions we have defined, and compute the whole way through:

In [198]:
# Apply Schur algorithm to triangularize a given real matrix A
# Can be modified slightly for complex matrix
# Delete all the print() function in the schur function if process need not be shown
# 5 digits precision is displayed
def schur(A):
    print('Schur decomposition on \n', A)
    print('\n\nStep 0')
    n = len(A)
    value, vector = eigenval_eigenvec(A)
    # Due to finite precision computing, there might be negligibly small funny imaginary part
    # Delete the next two lines if your matrix might have complex eigenvalues
    value = value.real
    vector = vector.real
    print('\nThe chosen eigenvector is ', np.round(vector, 5)) # change 5 to any number of decimal precision
    basis = extended_basis(vector)
    I1 = gram_schmidt(basis)
    print('\nI', 0, ' = \n', np.round(I1, 5))
    A1 = multiply(basis, A)
    print('\nA', 0, ' = \n', np.round(A1, 5))
    
    #Repeat the process
    for i in range(1, n - 1):
        print('\n\nStep ', i)
        B = np.zeros((n - i, n - i))
        for j in range(len(B)):
            B[j, :] = A1[j + i, i : n]
            
        print('\nThe submatrix is \n', np.round(B, 5))
        print()
        
        value, vector = eigenval_eigenvec(B)
        value = value.real
        vector = vector.real
        print('\nThe chosen eigenvector is ', np.round(vector, 5))
        I = extended_basis(vector)
        I = gram_schmidt(I)
        
        I2 = np.zeros((n, n))
        for j in range(i):
            I2[j][j] = 1
        for j in range(i, n):
            I2[j, i: n] = I[j - i, :]
        
        print('\nI', i, ' = \n', np.round(I2, 5))
        
        A1 = multiply(I2, A1)
        I1 = I1.dot(I2)
        print('\nA', i, ' = \n', np.round(A1, 5))

    return A1, I1

#C = [[-4, 2, 0, 12], [-1, 4, 0, 1], [-1, 0, 8, 2], [-6, 1, 0, 14]]
E = [[21.6, 3.2], [3.2, 26.4]]
T, Q = schur(A)
print('\n\nResult\n')
print('The resulting upper triangular matrix T = \n', np.round(T, 5))
print('\nQ = \n', np.round(I1, 5))
print('\nQ T Q^T = \n', np.round(I1.dot(A1).dot(np.transpose(Q)), 5))

Schur decomposition on 
 [[25. -1.  3.  1.]
 [ 1. 23. -1.  5.]
 [ 3.  1. 25. -1.]
 [-1.  5.  1. 23.]]


Step 0
The eigenvalues are  [28. 20. 20. 28.]
The eigenvectors are 
 [[ 0.70711  0.5      0.5     -0.13393]
 [ 0.       0.5      0.5      0.69431]
 [ 0.70711 -0.5     -0.5     -0.13393]
 [ 0.      -0.5     -0.5      0.69431]]

The chosen eigenvector is  [ 0.5  0.5 -0.5 -0.5]

I 0  = 
 [[ 0.5     -0.70711  0.40825 -0.28868]
 [ 0.5      0.      -0.8165  -0.28868]
 [-0.5      0.       0.      -0.86603]
 [-0.5     -0.70711 -0.40825  0.28868]]

A 0  = 
 [[20.      -2.82843  1.63299  2.3094 ]
 [ 0.      24.       2.3094   3.26599]
 [ 0.       2.3094  26.66667 -1.88562]
 [-0.       3.26599 -1.88562 25.33333]]


Step  1

The submatrix is 
 [[24.       2.3094   3.26599]
 [ 2.3094  26.66667 -1.88562]
 [ 3.26599 -1.88562 25.33333]]

The eigenvalues are  [20. 28. 28.]
The eigenvectors are 
 [[-0.70711  0.70711 -0.26491]
 [ 0.40825  0.40825 -0.90998]
 [ 0.57735  0.57735  0.31901]]

The chosen eig

We can check that $Q$ we got is indeed orthogonal by takeing its transpose and multiply together.

In [204]:
print('(Q^T)Q \n=', np.transpose(Q), Q, '\n=', np.round(np.transpose(Q).dot(Q), 5))

(Q^T)Q 
= [[ 0.5  0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5]
 [-0.5  0.5 -0.5  0.5]
 [ 0.5 -0.5 -0.5  0.5]] [[ 0.5 -0.5 -0.5  0.5]
 [ 0.5 -0.5  0.5 -0.5]
 [-0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5  0.5  0.5]] 
= [[ 1.  0.  0. -0.]
 [ 0.  1.  0. -0.]
 [ 0.  0.  1.  0.]
 [-0. -0.  0.  1.]]
